<h1>Space Debris - Exploratory Data Analysis</h1>
<h4>Blake Rayvid - <a href=https://github.com/brayvid>https://github.com/brayvid</a></h4>


Dataset: https://www.kaggle.com/datasets/kandhalkhandeka/satellites-and-debris-in-earths-orbit

![Sizes of Debris](https://i.pinimg.com/originals/50/2a/92/502a92f64945244e2a67a052f147573f.jpg)

[Image link](https://i.pinimg.com/originals/50/2a/92/502a92f64945244e2a67a052f147573f.jpg)

<h3>Dataset overview</h3>

This dataset contains roughly 14,000 objects, classified as `DEBRIS`, `PAYLOAD`, `ROCKET` or unknown,</br>and either `LARGE`, `MEDIUM`, `SMALL` or unknown.
</br></br>
According to Google\'s AI chatbot, space debris range from:</br>"Larger than 1 mm: 100 million \- 170 million objects</br>Larger than 1 cm: 670,000 objects</br>Larger than 10 cm: 25,000 \- 29,000 objects"</br></br>
<b>So we know this dataset does not contain all debris objects.</b></br>
</br>
We will investigate the object size and type columns, the orbital parameters, the country of origin and launch date.</br></br>

Note: This dataset does not contain any mass estimates.

The total mass of all space debris is known to exceed 9,000 metric tons. [Source](https://orbitaldebris.jsc.nasa.gov/faq/#:~:text=Return%20to%20Top-,3.,Earth%20exceeded%209%2C000%20metric%20tons.)


<h3>Read and clean data</h3>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt; plt.rcParams["figure.dpi"] = 144
import seaborn as sns
from matplotlib import ticker
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Check out the columns
df = pd.read_csv('/kaggle/input/satellites-and-debris-in-earths-orbit/space_decay.csv')
df.info()

In [ ]:
# Check out the values of important columns
df['RCS_SIZE'].unique()

In [ ]:
df['OBJECT_TYPE'].unique()

In [ ]:
df['COUNTRY_CODE'].unique()

In [ ]:
# Show how many elements of each column are NaN
df.isna().sum()

In [ ]:
# This column is always NaN so drop it entirely
df = df.drop(labels=['DECAY_DATE'], axis=1)

In [ ]:
 # Rename the object type to be more presentable
df['OBJECT_TYPE'] = df['OBJECT_TYPE'].replace(to_replace={'DEBRIS':'Debris','PAYLOAD':'Payload','TBA': 'Unknown','ROCKET BODY': 'Rocket'})
df['OBJECT_TYPE'].unique()

In [ ]:
 # Rename 'TBD' and NaN country codes to 'Unknown'
df['COUNTRY_CODE'] = df['COUNTRY_CODE'].replace(to_replace={'TBD':'Unknown',np.nan:'Unknown'})
df['COUNTRY_CODE'].unique()

In [ ]:
 # Rename object size for presentation
df['RCS_SIZE'] = df['RCS_SIZE'].replace(to_replace={'LARGE':'Large','MEDIUM':'Medium',"SMALL":"Small"})
df['RCS_SIZE'].unique()

In [ ]:
 # Create a new column for the period in hours
df['PERIOD_HOURS'] = df['PERIOD'] / 60

In [ ]:
 # Create new column for the (maximum) altitude above Earth surface (Earth radius ~ 6371km)
df['ALTITUDE_MI'] = (df['SEMIMAJOR_AXIS'] - 6371) * 0.6213

<h3>Descriptive questions</h3>

<h4>What time range does the dataset cover?</h4>

In [ ]:
df['AggValue'] = 1
has_launch_date_df = df.dropna(subset='LAUNCH_DATE')
has_launch_date_year_counts = has_launch_date_df.pivot_table(values='AggValue', index='LAUNCH_DATE', columns='OBJECT_TYPE', aggfunc=np.sum).cumsum()

fig, ax = plt.subplots(figsize=(4, 3))
plt.bar(x=has_launch_date_year_counts.index, height=has_launch_date_year_counts['Debris'])
plt.bar(x=has_launch_date_year_counts.index, height=has_launch_date_year_counts['Payload'], bottom=has_launch_date_year_counts['Debris'])
plt.bar(x=has_launch_date_year_counts.index, height=has_launch_date_year_counts['Rocket'], bottom=(has_launch_date_year_counts['Payload']+has_launch_date_year_counts['Debris']))
plt.legend(['Debris','Payload','Rocket'], framealpha=1, loc=(0.02,0.7))
plt.xlabel('Creation Year')
plt.ylabel('Count')
plt.title('Debris Objects Created')

# https://stackoverflow.com/questions/25973581/how-to-format-axis-number-format-to-thousands-with-a-comma
thousands_format = ticker.StrMethodFormatter('{x:,.0f}')
ax.yaxis.set_major_formatter(thousands_format)
plt.show()

<h4>How much of each size and type of space debris is there?</h4>

In [ ]:
# Plot heatmap of size vs type
df_filled = df.copy()
df_filled.loc[:,'RCS_SIZE'] = df_filled['RCS_SIZE'].fillna('Unknown')
class_counts = df_filled.pivot_table(values='AggValue', index='RCS_SIZE', columns='OBJECT_TYPE', aggfunc=np.sum)

fig, ax = plt.subplots(figsize=(4, 3))
sns.heatmap(class_counts, cmap="Blues", annot=True, fmt='g')
plt.ylabel('Size')
plt.xlabel('Type')
plt.title("Object Count by Size and Type")
plt.yticks(rotation=0)
plt.show()

<h4>In what orbits are the debris mainly located?</h4>

In [ ]:
# Display statistics of the orbital elements
debris_elements_df = df[['ECCENTRICITY','INCLINATION', 'RA_OF_ASC_NODE','ARG_OF_PERICENTER', 'MEAN_ANOMALY','SEMIMAJOR_AXIS', 'PERIOD_HOURS', 'APOAPSIS', 'PERIAPSIS']]
debris_elements_df.describe()

In [ ]:
# Plot count vs altitude histogram for just large debris
PERIOD_NO_OUTLIERS_LARGE_MASK = ((df['PERIOD_HOURS'] < 27) & (df['RCS_SIZE'] == 'Large'))
period_no_outliers_large_df = df[PERIOD_NO_OUTLIERS_LARGE_MASK]

fig, ax = plt.subplots(figsize=(4, 3))
sns.histplot(data=period_no_outliers_large_df, x="ALTITUDE_MI",bins=30)
plt.xlabel('Altitude (mi)')
plt.title('Distribution of Large Debris by Altitude')
ax.yaxis.set_major_formatter(thousands_format)
ax.xaxis.set_major_formatter(thousands_format)
plt.show()

In [ ]:
# Inclination vs period scatter plot
sizes = ['Large', 'Medium', 'Small']
sizes_palette = {'Small': 'green', 'Medium':'orange', 'Large': 'red'}

fig, ax = plt.subplots(figsize=(4, 3))
sns.scatterplot(x=df['PERIOD_HOURS'], y=df['INCLINATION'],s=10, size=df['RCS_SIZE'], size_order=sizes, sizes=[18,16,10], hue=df['RCS_SIZE'], palette=sizes_palette, hue_order=sizes)
plt.xlabel('Period (hours)')
plt.ylabel('Inclination (degrees)')
plt.title('Inclination vs. Period of Space Debris')
plt.xlim((0.75,28.5))
plt.ylim((-2,105))
plt.legend()
plt.show()

Notice the vertical bands at roughly T= 1.5h and T = 24h. This corresponds to the lowest-earth orbits and geosynchronous orbit respectively.

In [ ]:
# Eccentricity vs period scatter plot
fig, ax = plt.subplots(figsize=(4, 3))
sns.scatterplot(x=df['PERIOD_HOURS'], y=df['ECCENTRICITY'],s=10, size=df['RCS_SIZE'], size_order=sizes, sizes=[18,16,10], hue=df['RCS_SIZE'], palette=sizes_palette, hue_order=sizes)
plt.xlabel('Period (hours)')
plt.ylabel('Eccentricity')
plt.title('Eccentricity vs. Period of Space Debris')
plt.xlim((0.75,28.5))
plt.ylim((-0.02,0.86))
plt.legend()
plt.show()

In [ ]:
# Inclination vs eccentricity scatter plot
fig, ax = plt.subplots(figsize=(4, 3))
sns.scatterplot(x=df['ECCENTRICITY'], y=df['INCLINATION'],s=10, size=df['RCS_SIZE'], size_order=sizes, sizes=[18,16,10], hue=df['RCS_SIZE'], palette=sizes_palette, hue_order=sizes)
plt.xlabel('Eccentricity')
plt.ylabel('Inclination (degrees)')
plt.title('Inclination vs. Eccentricity of Space Debris')
plt.xlim((-0.02,0.88))
plt.ylim((-2,105))
plt.legend()
plt.show()

<h4>Which countries are responsible for the most debris?</h4>

In [ ]:
# Bar chart count by country
top_counts = df['COUNTRY_CODE'].value_counts()[:5]

fig, ax = plt.subplots(figsize=(4, 3))
top_counts.plot(kind='bar')
plt.xlabel('Country of Origin')
plt.ylabel('Count')
plt.title('Countries Responsible For Most Space Debris')
plt.xticks(ticks=[0,1,2,3,4], labels=["US",'China','Russia','UK','France'],rotation=0)
ax.yaxis.set_major_formatter(thousands_format)
plt.show()

In [ ]:
responsibility = top_counts / len(df)
print(f'The US is responsible for roughly {int(100 * responsibility[0])}% of space debris.')

<h4>How many small, medium and large objects are there from the top 3 contributing countries?</h4>

In [ ]:
# Just look at top 3 contributors
top_codes = top_counts.keys()[:3]
TOP_MASK = (df['COUNTRY_CODE'].isin(top_codes))
top_df = df[TOP_MASK]
country_names = ["US",'China','Russia']

country_counts = top_df.pivot_table(values='AggValue', index='RCS_SIZE', columns='COUNTRY_CODE', aggfunc=np.sum)

# Specify order of rows and columns
country_counts = country_counts.reindex(index=sizes, columns=top_codes)

fig, ax = plt.subplots(figsize=(4, 3))
sns.heatmap(country_counts, cmap="Blues", annot=True, fmt='g')
plt.ylabel('Size')
plt.xlabel('Country')
plt.title("Object Count by Size and Country of Origin")
plt.xticks(ticks=[0.5,1.5,2.5], labels=country_names)
plt.yticks(ticks=[0.5,1.5,2.5], labels=sizes,rotation=0)
plt.show()

<h4>How many of each category of object are there from the top 3 contributing countries?</h4>

In [ ]:
top_codes = top_counts.keys()[:3]
TOP_MASK = (df_filled['COUNTRY_CODE'].isin(top_codes))
top_df = df[TOP_MASK]

country_counts = top_df.pivot_table(values='AggValue', index='OBJECT_TYPE', columns='COUNTRY_CODE', aggfunc=np.sum)

# Specify order of rows and columns
obj_types = ['Debris', 'Payload', 'Rocket','Unknown']
country_counts = country_counts.reindex(index=obj_types, columns=top_codes)

fig, ax = plt.subplots(figsize=(4, 3))
sns.heatmap(country_counts, cmap="Blues", annot=True, fmt='g')
plt.ylabel('Type')
plt.xlabel('Country')
plt.title("Object Count by Type and Country of Origin")
plt.xticks(ticks=[0.5,1.5,2.5], labels=country_names)
plt.yticks(ticks=[0.5,1.5,2.5,3.5], labels=obj_types, rotation=0)
plt.show()

<h4>How many objects' country of origin is unknown?</h4>

In [ ]:
NO_COUNTRY_MASK = ((df['COUNTRY_CODE'] == 'Unknown'))
no_country_df = df[NO_COUNTRY_MASK]
print(f"There are {len(no_country_df)} debris objects in this dataset with unknown country of origin.")

In [ ]:
no_country_df.loc[:,'RCS_SIZE'] = no_country_df['RCS_SIZE'].fillna('Unknown')
no_country_class_counts = no_country_df.pivot_table(values='AggValue', index='RCS_SIZE', columns='OBJECT_TYPE', aggfunc=np.sum)

fig, ax = plt.subplots(figsize=(4, 3))
sns.heatmap(no_country_class_counts, cmap="Blues", annot=True, fmt='g')
plt.ylabel('Size')
plt.xlabel('Type')
plt.title("Objects With Unknown Country of Origin")
plt.xticks(ticks=[0.5,1.5], labels=["Payload", 'Unknown'])
plt.yticks(ticks=[0.5,1.5,2.5,3.5], labels=["Large",'Medium','Small', 'Unknown'],rotation=0)
plt.show()

<h4>Dive into specific slices and analyze their orbits.</h4>

In [ ]:
AMERICAN_LARGE_MASK = ((df['COUNTRY_CODE'] == 'US') & (df['RCS_SIZE'] == 'Large') & (df['OBJECT_TYPE'] != 'Unknown')) # exclude objects of unknown type
american_large_df = df[AMERICAN_LARGE_MASK]

obj_type_2 = ['Debris','Payload','Rocket']

fig, ax = plt.subplots(figsize=(4, 3))
sns.scatterplot(x=american_large_df['ALTITUDE_MI'], y=american_large_df['INCLINATION'],s=20,hue=american_large_df['OBJECT_TYPE'],hue_order=obj_type_2)
plt.xlabel('Altitude (mi)')
plt.ylabel('Inclination (degrees)')
plt.title('Large American Space Debris')
plt.xlim((-300, 23500))
plt.ylim((-2,102))
plt.legend()
ax.xaxis.set_major_formatter(thousands_format)
plt.show()

In [ ]:
CHINESE_LARGE_MASK = ((df['COUNTRY_CODE'] == 'PRC') & (df['RCS_SIZE']=='Large') & (df['OBJECT_TYPE'] != 'Unknown')) # exclude objects of unknown type
chinese_large_df = df[CHINESE_LARGE_MASK]

fig, ax = plt.subplots(figsize=(4, 3))
sns.scatterplot(x=chinese_large_df['ALTITUDE_MI'], y=chinese_large_df['INCLINATION'],s=20,hue=chinese_large_df['OBJECT_TYPE'],hue_order=obj_type_2)
plt.xlabel('Altitude (mi)')
plt.ylabel('Inclination (degrees)')
plt.title('Large Chinese Space Debris')
plt.xlim((-300, 24000))
plt.ylim((-3,105))
plt.legend()
ax.xaxis.set_major_formatter(thousands_format)
plt.show()

In [ ]:
LEO_MASK = (df['PERIOD'] <= 130)
leo_df = df[LEO_MASK]

US_CN_RU_MASK = (leo_df['COUNTRY_CODE'].isin(['US','PRC','CIS','Unknown']))
leo_ucr_df = leo_df[US_CN_RU_MASK]

LG_MASK = (leo_ucr_df['RCS_SIZE'] == 'Large')
leo_ucr_lg_df = leo_ucr_df[LG_MASK]

fig, ax = plt.subplots(figsize=(4, 3))
sns.scatterplot(x=leo_ucr_lg_df['ALTITUDE_MI'], y=leo_ucr_lg_df['INCLINATION'],s=20, hue=leo_ucr_lg_df['COUNTRY_CODE'], hue_order=['US','PRC','CIS','Unknown'])
plt.xlabel('Altitude (mi)')
plt.ylabel('Inclination (degrees)')
plt.title('Large Low-Earth-Orbit Debris')
plt.ylim((-2.5,104))

# https://stackoverflow.com/questions/23037548/change-main-plot-legend-label-text
handles, previous_labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=['US','China','Russia','Unknown'])
ax.xaxis.set_major_formatter(thousands_format)
plt.show()

<h4>Recommendations</h4>

<ul>
<li>Invest in R&D for more robust tracking of smaller objects.
<li>Develop space debris removal solutions like active capture and laser ablative steering.
<li>Adopt sustainable satellite and rocket design practices.
</ul>